Here is the code to release the queries with the parameters you have selected using the OpenDP library

In [ ]:
# !pip install opendp

In [ ]:
# notebook imports
import opendp.prelude as dp
import math
import pandas as pd
import numpy as np
dp.enable_features("contrib")
dp.enable_features("honest-but-curious")
from typing import List

## Query: workclass_histogram

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/adult_income.csv')

In [ ]:
query_info = {"export_selected_query": "workclass_histogram", "export_mechanism": "laplace", "export_epsilon": 0.25, "url": "https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/adult_income.csv", "export_data_type": "categorical"}

In [ ]:
def at_delta(meas, delta):
    meas = dp.c.make_zCDP_to_approxDP(meas)
    return dp.c.make_fix_delta(meas, delta)

In [ ]:
# Load data and get parameters
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/adult_income.csv')
query_type = 'workclass_histogram'
mechanism = 'laplace'
epsilon = 0.25
column_name = 'workclass'
categories = [' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay']
column_type = df[column_name].dtype

In [ ]:
histogram = (
dp.t.make_split_dataframe(separator=",", col_names=list(df.columns)) >>
dp.t.make_select_column(column_name, str) >>
dp.t.then_count_by_categories(categories=[c[1:] for c in categories], MO=dp.L1Distance[int] )
)
noisy_laplace_histogram = dp.binary_search_chain(
lambda s: histogram >> dp.m.then_laplace(scale=s),
d_in=1, d_out=epsilon)
sensitive_counts = noisy_laplace_histogram(df.to_csv(index=False, header=False))

In [ ]:
print("Noisy Laplace Counts:", sensitive_counts[:-1])